In [3]:
import keras

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from imblearn.over_sampling import SMOTE 
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
train=pd.read_csv("84497f2b3a53ac1bff2d67db0ba4a2a6.csv")
test=pd.read_csv("4904b5a6417f2c49badf34dbf4dafdc5.csv")
train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Argentina,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Argentina,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Argentina,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Argentina,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Argentina,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


### tratamiento simple de los datos

In [5]:
#porcentaje de missing
round(100*(train.isnull().sum()/len(train.index)), 2)
#la base viene sin missing
def variables_dummy(dataset,variable):
    cat_dummies = pd.get_dummies(dataset[variable],prefix=variable).iloc[:,1:]
    data_new = dataset.join(cat_dummies)
    dataset = data_new.drop([variable],axis=1)
    return dataset
muestra = train
muestra1=variables_dummy(muestra,"bank_account")
muestra1=variables_dummy(muestra1,"location_type")
muestra1=variables_dummy(muestra1,"cellphone_access")
muestra1=variables_dummy(muestra1,"gender_of_respondent")
muestra1=variables_dummy(muestra1,"relationship_with_head")
muestra1=variables_dummy(muestra1,"marital_status")
muestra1=variables_dummy(muestra1,"education_level")
muestra1=variables_dummy(muestra1,"job_type")
muestra1=variables_dummy(muestra1,"country")
train=muestra1
train["household_size"] = np.where(train["household_size"] > train["household_size"].quantile(0.95), train["household_size"].mean(), train["household_size"])
train["age_of_respondent"] = np.where(train["age_of_respondent"] < train["age_of_respondent"].quantile(0.05), train["age_of_respondent"].mean(), train["age_of_respondent"])
muestra = test
muestra1=variables_dummy(muestra,"location_type")
muestra1=variables_dummy(muestra1,"cellphone_access")
muestra1=variables_dummy(muestra1,"gender_of_respondent")
muestra1=variables_dummy(muestra1,"relationship_with_head")
muestra1=variables_dummy(muestra1,"marital_status")
muestra1=variables_dummy(muestra1,"education_level")
muestra1=variables_dummy(muestra1,"job_type")
muestra1=variables_dummy(muestra1,"country")
test=muestra1
y=train["bank_account_Yes"]
x=train.drop(["bank_account_Yes","year","uniqueid"], axis=1)

### nivel de conversion

In [6]:
conversion = (sum(train['bank_account_Yes'])/len(train['bank_account_Yes'].index))*100
conversion
#14%

14.079238224791702

#### Separamos muestras

In [7]:
x_res,x_test,y_res,y_test=train_test_split(x,y,test_size=0.2,random_state=1010,stratify=y)

#### Balanceo de clases solo para entrenamiento

In [10]:
nm = SMOTE(random_state = 10101)
x_train,y_train=nm.fit_resample(x_res,y_res)

#### Definimos modelos

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


logistico_grid={"C":np.logspace(-3,3,3), "penalty":["l1","l2"],"solver":["liblinear","lbfgs"]}

rf_grid = {'n_estimators':[25,30,50,80],'max_depth':[8,10,15],'random_state': [0,1,5]}

lr=GridSearchCV(LogisticRegression(),logistico_grid,cv=5,scoring='roc_auc') 

randf=GridSearchCV(RandomForestClassifier(),rf_grid, cv=5,scoring='roc_auc')

#### Ajustamos

In [20]:
ajuste_lr=lr.fit(x_train,y_train)

In [ ]:
ajuste_rf=randf.fit(x_train,y_train)

### vemos el nivel de ajuste

In [22]:
accuracy_score(y_test,ajuste_lr.predict(x_test))

0.8184909670563231

In [ ]:
accuracy_score(y_test,ajuste_rf.predict(x_test))

### ajustemos por un modelos staking

In [ ]:
from sklearn.ensemble import StackingClassifier


modelos =[
    ('lr',LogisticRegression(C=1,penalty="l2",solver="liblinear")),
    ('rfc',RandomForestClassifier(max_depth=20,n_estimators=85,random_state=0))
         ]

In [ ]:
modelo_salida = RandomForestClassifier(max_depth=20,n_estimators=85,random_state=0)


#voy a forzarlo a usar las variables predictivas con las predichas

stack1=StackingClassifier(estimators=modelos, final_estimator=modelo_salida,n_jobs=1,passthrough=True,verbose=10)

stack1.fit(x_train,y_train)

In [ ]:
accuracy_score(y_test,stack1.predict(x_test))